## ST (Pruebas)

* Optimización inferencia y tiempos de respuesta

In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import logging
import torch
import textstat
import nltk
import time
import re
import yaml # Importamos PyYAML

# Importaciones específicas de Code 1 para LLMs y Azure
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import AzureOpenAI
import google.generativeai as genai
from bert_score import score as bert_score
from google.generativeai.types import GenerationConfig

# --- Cargar configuración desde YAML ---
try:
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    print("Configuración cargada exitosamente desde config.yaml")
except FileNotFoundError:
    print("Error: 'config.yaml' no encontrado. Asegúrate de que el archivo exista en el mismo directorio.")
    exit()
except yaml.YAMLError as e:
    print(f"Error al parsear 'config.yaml': {e}")
    exit()

# Temperatura del asistente de creación de preguntas y respuestas
MD_GEN = "GEMINI"
modelo_banco_generacion = "models/gemini-1.5-pro-latest"
temperature_gen_assitant = 0.7

# Asignar prompts y parámetros desde el archivo de configuración
prompt_systemE1 = config['prompts']['system_prompt_E1']
prompt_systemE2 = config['prompts']['system_prompt_E2']
prompt_generacion_qa = config['prompts']['qa_generation_prompt']

# Parámetros desde el YAML
num_pares_a_generar = config['parameters']['num_qa_to_generate']
chunk_text = config['parameters']['chunk_size']
chunk_over = config['parameters']['chunk_overlap']
top_join = config['parameters']['top_join_chunks'] # Este es el 'top_join' para la búsqueda semántica
max_attempts = config['parameters']['max_api_attempts']
wait_time_on_error_s = config['parameters']['api_wait_time_on_error_s']
temperature_ = config['parameters']['gpt_temperature']
temperature_2 = config['parameters']['gemma_temperature']
perfil_usuario = config['parameters']['user_profile']
contexto_generico = config['parameters']['generic_context']

# Umbrales desde el YAML
umbrales = [
    config['thresholds']['similitud_min'],
    config['thresholds']['coherencia_min'],
    config['thresholds']['personalizacion_min'],
    config['thresholds']['fluidez_min'],
    config['thresholds']['alucinacion_min']
]

#################################################
############## Modelos a Comparar ###############
#################################################

#################### modelo 1 ###################
model_embedding = "text-embedding-ada-002"
#modelo_banco = "gpt-35-turbo-16k-PQR"
modelo_banco = "gpt-4o-mini_clasificacion-PQRS"

################### modelo 2 ####################
model_embedding2 = "models/text-embedding-004"
#modelo_banco2 = "models/gemma-3-27b-it"
modelo_banco2 = "models/gemini-2.5-flash"

# --- Configuración inicial ---
nltk.download('punkt', quiet=True)
warnings.filterwarnings("ignore")
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

def medir_coherencia(pregunta, respuesta):
    """Mide la coherencia entre una pregunta y su respuesta utilizando BERTScore."""
    if not respuesta.strip() or not pregunta.strip():
        return 0.0
    # Utiliza 'en' como idioma para BERTScore con 'roberta-base'
    P, R, F1 = bert_score([respuesta], [pregunta], lang="en", model_type="roberta-base", rescale_with_baseline=False)
    return float(F1[0])

def medir_personalizacion(perfil_usuario, respuesta):
    """Mide qué tan personalizada es una respuesta en relación con un perfil de usuario."""
    if not respuesta.strip() or not perfil_usuario.strip():
        return 0.0
    P, R, F1 = bert_score([respuesta], [perfil_usuario], lang="en", model_type="roberta-base", rescale_with_baseline=False)
    return float(F1[0])

def medir_fluidez(texto):
    """Mide la fluidez de un texto utilizando el índice de facilidad de lectura de Flesch."""
    if not texto.strip():
        return 0.0
    return textstat.flesch_reading_ease(texto)

def medir_alucinacion(respuesta, contexto_fuente):
    """Mide la alucinación comparando la respuesta con un contexto fuente (usando BERTScore)."""
    if not respuesta.strip() or not contexto_fuente.strip():
        return 0.0
    P, R, F1 = bert_score([respuesta], [contexto_fuente], lang="en", model_type="roberta-base", rescale_with_baseline=False)
    return float(F1[0])

############## Modelo 1 (Azure OpenAI) #################
# 1. Cargar credenciales desde JSON
def cargar_credenciales(ruta_credenciales):
    with open(ruta_credenciales, "r") as file:
        return json.load(file)

# 2. Crear cliente AzureOpenAI
def crear_cliente_azure(creds):
    client = AzureOpenAI(
        api_key=creds["AZURE_API_KEY"],
        api_version=creds["AZURE_API_VERSION"],
        azure_endpoint=creds["AZURE_ENDPOINT"]
    )
    return client

# 3. Generar embeddings usando cliente (AzureOpenAI)
def text_embedding(text):
    input_text = [text] if isinstance(text, str) else text
    try:
        embeddings = client.embeddings.create(model=model_embedding,
                                              input=input_text,
                                              encoding_format="float")
        return embeddings.data[0].embedding
    except Exception as e:
        print(f"Error al generar embedding con Azure: {e}")
        return np.zeros(1536) # Devuelve un array de ceros en caso de error

# 4. Inicialización Global del Cliente Azure y Credenciales
try:
    ruta_credenciales = "credentials.json"
    creds = cargar_credenciales(ruta_credenciales)
    client = crear_cliente_azure(creds)
    print("Cliente Azure OpenAI inicializado.")
except Exception as e:
    print(f"Error al inicializar cliente Azure OpenAI: {e}")
    client = None # Asegurarse de que el cliente es None si falla la inicialización

############## Modelo 2 (Gemini/Gemma) ################
def cargar_credenciales2(ruta_credenciales):
    with open(ruta_credenciales, "r") as file:
        return json.load(file)

try:
    ruta_credenciales2 = "credentials2.json"
    creds2 = cargar_credenciales2(ruta_credenciales2)
    genai.configure(api_key=creds2["GEMINI_API_KEY"])
    print("Cliente Gemini/Gemma configurado.")
except Exception as e:
    print(f"Error al configurar cliente Gemini/Gemma: {e}")
    genai = None # Asegurarse de que genai es None si falla la configuración

def text_embedding_gemma(text):
    if genai is None:
        print("Error: El cliente Gemini/Gemma no está configurado para embeddings.")
        return np.zeros(768)
    try:
        result = genai.embed_content(model=model_embedding2,
                                     content=text,
                                     task_type="RETRIEVAL_QUERY") # O RETRIEVAL_DOCUMENT
        return result['embedding']
    except Exception as e:
        print(f"Error al generar embedding con Gemma: {e}")
        return np.zeros(768) # Devuelve un array de ceros en caso de error

# --- Carga y chunking del PDF ---
print("Cargando y procesando PDF...")
try:
    loader = PyPDFLoader("llm_doc.pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_text, chunk_overlap=chunk_over)
    doc_splits = text_splitter.split_documents(documents)

    data = [{'Chunks': doc.page_content, 'Metadata': doc.metadata} for doc in doc_splits]
    df_vector_store = pd.DataFrame(data)

    df_vector_store_gpt = df_vector_store.copy()
    df_vector_store_gem = df_vector_store.copy()

    # Generar embeddings para GPT
    if client:
        print("Generando embeddings para GPT (Azure)...")
        df_vector_store_gpt["Embedding"] = df_vector_store_gpt["Chunks"].apply(lambda x: text_embedding(x))
        df_vector_store_gpt["Embedding"] = df_vector_store_gpt["Embedding"].apply(np.array)
    else:
        print("Saltando embeddings para GPT, cliente Azure no inicializado.")
        df_vector_store_gpt["Embedding"] = [np.zeros(1536)] * len(df_vector_store_gpt)

    # Generar embeddings para Gemma
    if genai:
        print("Generando embeddings para Gemma...")
        df_vector_store_gem["Embedding"] = df_vector_store_gem["Chunks"].apply(text_embedding_gemma)
        df_vector_store_gem["Embedding"] = df_vector_store_gem["Embedding"].apply(np.array)
    else:
        print("Saltando embeddings para Gemma, cliente Gemini/Gemma no configurado.")
        df_vector_store_gem["Embedding"] = [np.zeros(768)] * len(df_vector_store_gem)

except Exception as e:
    print(f"Error durante la carga y procesamiento del PDF: {e}")
    exit()

# --- Cargar preguntas_BOT.xlsx para evitar duplicados (si existe) ---
try:
    df_preguntas_respuestas_existentes = pd.read_excel('preguntas_BOT.xlsx')
except FileNotFoundError:
    print("Advertencia: 'preguntas_BOT.xlsx' no encontrado. Se asumirá que no hay preguntas existentes.")
    df_preguntas_respuestas_existentes = pd.DataFrame(columns=["Pregunta", "Respuesta"])

# --- FUNCIÓN PARA GENERAR UN CONTEXTO A PARTIR DE CHUNKS SELECCIONADOS ---
def obtener_contexto_para_generacion(df_vector_store, num_chunks=5):
    if len(df_vector_store) < num_chunks:
        # Si no hay suficientes chunks, usa todos los disponibles
        return "\n".join(df_vector_store['Chunks'].tolist())
    else:
        # Distribuye los chunks equitativamente
        indices = np.linspace(0, len(df_vector_store) - 1, num_chunks, dtype=int)
        selected_chunks = df_vector_store.loc[indices, 'Chunks'].tolist()
        return "\n".join(selected_chunks)

# --- GENERACIÓN DE NUEVAS PREGUNTAS Y RESPUESTAS ---
nuevos_pares_qa = []

# Corregido: Usar 'top_join' para la generación de QA, que ahora proviene del YAML

if MD_GEN == "GPT":
    contexto_para_generacion = obtener_contexto_para_generacion(df_vector_store_gem, 
                                                            num_chunks=top_join)
else:
    contexto_para_generacion = obtener_contexto_para_generacion(df_vector_store_gpt, 
                                                            num_chunks=top_join)

# Formatear el prompt de generación con el número deseado (ahora `num_pares_a_generar` del YAML)
final_prompt_qa = prompt_generacion_qa.format(
    source_text=contexto_para_generacion,
    num_to_generate=num_pares_a_generar
)

print(f"Enviando solicitud al LLM para generar {num_pares_a_generar} pares QA...")
try:
    if client: # Solo intentar si el cliente Azure está inicializado
        if MD_GEN == "GPT":
            completion_qa = client.chat.completions.create(
            model=modelo_banco_generacion,
            temperature=temperature_gen_assitant, # Usa la temperatura de GPT del YAML
            messages=[
                {"role": "system", "content": final_prompt_qa}
            ]
            )
            generated_text = completion_qa.choices[0].message.content
        else:
            model_2 = genai.GenerativeModel(model_name=modelo_banco_generacion)
            full_query_for_gemma = final_prompt_qa
            completion2 = model_2.generate_content(
            contents=full_query_for_gemma,
            generation_config=GenerationConfig(temperature=temperature_gen_assitant)
            )
            generated_text = completion2.text


        print("Generación del LLM completada. Procesando pares QA...")

        # --- PARSEO DE LAS RESPUESTAS GENERADAS CON REGEX (PARA MÚLTIPLES PARES) ---
        pattern = re.compile(r"Pregunta:\s*(.*?)\s*Respuesta:\s*(.*?)(?=(?:Pregunta:|$))", re.DOTALL | re.IGNORECASE)
        matches = pattern.findall(generated_text)

        for match in matches:
            question = match[0].strip()
            answer = match[1].strip()

            if question and answer: # Asegurarse de que no estén vacíos
                # Validar si ya existe para evitar duplicados
                if not df_preguntas_respuestas_existentes['Pregunta'].astype(str).str.contains(question, case=False, regex=False).any():
                    nuevos_pares_qa.append({"Pregunta": question, "Respuesta": answer})
                else:
                    print(f"  [QA Gen] Par '{question[:30]}...' ya existe en 'preguntas_BOT.xlsx', omitiendo.")

                if len(nuevos_pares_qa) >= num_pares_a_generar:
                    break # Detener si ya generamos la cantidad deseada

    else:
        print("Error: El cliente Azure OpenAI no está inicializado. No se pudo generar QA.")

except Exception as e:
    print(f"Error durante la generación o procesamiento de QA: {e}")

# Asegurarse de tener a lo sumo num_pares_a_generar
if len(nuevos_pares_qa) > num_pares_a_generar:
    nuevos_pares_qa = nuevos_pares_qa[:num_pares_a_generar]
elif len(nuevos_pares_qa) < num_pares_a_generar:
    print(f"Advertencia: Solo se pudieron generar {len(nuevos_pares_qa)} pares QA válidos de {num_pares_a_generar} solicitados.")

df_nuevos_qa = pd.DataFrame(nuevos_pares_qa)

if df_nuevos_qa.empty:
    print("No se pudieron generar pares de preguntas y respuestas válidos. Finalizando.")
    exit()

def cosine_similarity_for_chunks(row_embedding_chunk, query_vector):
    denominator1 = np.linalg.norm(row_embedding_chunk)
    denominator2 = np.linalg.norm(query_vector)
    dot_prod = np.dot(row_embedding_chunk, query_vector)
    if denominator1 == 0 or denominator2 == 0:
        return 0.0
    return dot_prod / (denominator1 * denominator2)

def cosine_final(v1, v2):
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    if norm_v1 == 0 or norm_v2 == 0:
        return 0.0
    return np.dot(v1, v2) / (norm_v1 * norm_v2)

def get_context_from_query(query, vector_store, n_chunks, embedding_func_for_query):
    # Usar la función de embedding pasada para generar el embedding de la consulta
    query_embedding_val = embedding_func_for_query(query)
    query_vector = np.array(query_embedding_val)

    # Calcular la similitud de coseno
    # La función lambda debe capturar query_vector correctamente
    top_matched_indices = (
        vector_store["Embedding"]
        .apply(lambda x: cosine_similarity_for_chunks(x, query_vector))
        .sort_values(ascending=False)[:n_chunks]
        .index
    )
    top_matched_df = vector_store[vector_store.index.isin(top_matched_indices)][["Chunks"]]
    return list(top_matched_df['Chunks'])

resultados, resultados2 = [], []
df_respuestas_detalladas = [] # Lista para almacenar los detalles de todas las respuestas

# Los prompts ahora se cargan desde el YAML, pero se usan en un bucle para la evaluación
prompts_evaluacion = [prompt_systemE1, prompt_systemE2]
# Nombres de los prompts para identificarlos en el DataFrame de resultados
prompt_names = ["prompt_systemE1", "prompt_systemE2"]

for idx, prompt_system in enumerate(prompts_evaluacion):
    current_prompt_name = prompt_names[idx] # Obtener el nombre del prompt actual

    for i, row in df_nuevos_qa.iterrows():
        pregunta = str(row["Pregunta"])
        respuesta_humana = str(row["Respuesta"])

        # --- Evaluación para GPT (Modelo 1) ---
        if client: # Solo ejecutar si el cliente Azure está inicializado
            respuesta_modelo_gpt = "No se pudo generar respuesta."
            input_tokens_gpt = 0
            output_tokens_gpt = 0
            api_latency_gpt = 0
            total_exec_time_gpt = 0
            successful_gpt_response = False
            contexto_gpt_usado = ""

            for attempt in range(max_attempts):
                try:
                    start_time_total = time.time()
                    context_chunks_gpt = get_context_from_query(pregunta, df_vector_store_gpt, top_join, text_embedding)
                    contexto_gpt_usado = " ".join(context_chunks_gpt) # Para alucinación
                    custom_prompt_gpt = prompt_system.format(source=str(context_chunks_gpt))

                    llm_start_time = time.time()
                    completion = client.chat.completions.create(
                        model=modelo_banco,
                        temperature=temperature_,
                        messages=[
                            {"role": "system", "content": custom_prompt_gpt},
                            {"role": "user", "content": pregunta}
                        ]
                    )
                    respuesta_modelo_gpt = completion.choices[0].message.content
                    llm_end_time = time.time()

                    input_tokens_gpt = completion.usage.prompt_tokens
                    output_tokens_gpt = completion.usage.completion_tokens
                    api_latency_gpt = llm_end_time - llm_start_time
                    total_exec_time_gpt = time.time() - start_time_total
                    successful_gpt_response = True
                    break # Salir del bucle de reintentos si fue exitoso
                except Exception as e:
                    print(f"    [GPT] Error en intento {attempt + 1}/{max_attempts} para '{pregunta[:50]}...': {e}")
                    if "quota" in str(e).lower() or "rate limit" in str(e).lower():
                        print(f"    [GPT] Error de cuota. Esperando {wait_time_on_error_s}s...")
                        time.sleep(wait_time_on_error_s)
                    else:
                        break # Otros errores no reintentables

            # Calcular métricas para GPT
            similitud_gpt = cosine_final(np.array(text_embedding(respuesta_modelo_gpt)), np.array(text_embedding(respuesta_humana))) if successful_gpt_response else 0.0
            coherencia_gpt = medir_coherencia(pregunta, respuesta_modelo_gpt) if successful_gpt_response else 0.0
            personalizacion_gpt = medir_personalizacion(perfil_usuario, respuesta_modelo_gpt) if successful_gpt_response else 0.0
            fluidez_gpt = medir_fluidez(respuesta_modelo_gpt) if successful_gpt_response else 0.0
            alucinacion_gpt = medir_alucinacion(respuesta_modelo_gpt, contexto_gpt_usado) if successful_gpt_response else 0.0

            df_respuestas_detalladas.append({
                "Modelo": modelo_banco,
                "Prompt_Tipo": prompt_system, # Usamos el nombre del prompt
                "Pregunta": pregunta,
                "Respuesta_Humana": respuesta_humana,
                "Respuesta_Modelo": respuesta_modelo_gpt,
                "Similitud_Coseno": similitud_gpt,
                "Coherencia": coherencia_gpt,
                "Personalizacion": personalizacion_gpt,
                "Fluidez": fluidez_gpt,
                "Alucinacion": alucinacion_gpt,
                "Input_Tokens": input_tokens_gpt,
                "Output_Tokens": output_tokens_gpt,
                "API_Latency_s": api_latency_gpt,
                "Total_Exec_Time_s": total_exec_time_gpt,
                "Successful_Response": successful_gpt_response
            })
        else:
            print(f"Saltando evaluación de GPT para pregunta '{pregunta[:50]}...', cliente no inicializado.")


        # --- Evaluación para GEMINI (Modelo 2) ---
        if genai: # Solo ejecutar si el cliente Gemini/Gemma está configurado
            respuesta_modelo_gemma = "No se pudo generar respuesta."
            input_tokens_gemma = 0
            output_tokens_gemma = 0
            api_latency_gemma = 0
            total_exec_time_gemma = 0
            successful_gemma_response = False
            contexto_gemma_usado = ""

            for attempt in range(max_attempts):
                try:
                    start_time_total = time.time()
                    context_chunks_gemma = get_context_from_query(pregunta, df_vector_store_gem, top_join, text_embedding_gemma)
                    contexto_gemma_usado = " ".join(context_chunks_gemma) # Para alucinación
                    custom_prompt_gemma = prompt_system.format(source=str(context_chunks_gemma))
                    model_2 = genai.GenerativeModel(model_name=modelo_banco2)
                    full_query_for_gemma = custom_prompt_gemma + "\n\nPregunta del usuario: " + pregunta

                    llm_start_time = time.time()
                    completion2 = model_2.generate_content(
                        contents=full_query_for_gemma,
                        generation_config=GenerationConfig(temperature=temperature_2)
                    )
                    respuesta_modelo_gemma = completion2.text
                    llm_end_time = time.time()

                    # Para Gemini/Gemma, los tokens son una estimación simple si no se exponen directamente en la respuesta
                    input_tokens_gemma = len(full_query_for_gemma.split())
                    output_tokens_gemma = len(respuesta_modelo_gemma.split())
                    api_latency_gemma = llm_end_time - llm_start_time
                    total_exec_time_gemma = time.time() - start_time_total
                    successful_gemma_response = True
                    break # Salir del bucle de reintentos si fue exitoso
                except Exception as e:
                    print(f"    [GEMINI] Error en intento {attempt + 1}/{max_attempts} para '{pregunta[:50]}...': {e}")
                    if "ResourceExhausted" in str(e) or "quota" in str(e).lower() or "rate limit" in str(e).lower():
                        print(f"    [GEMINI] Error de cuota. Esperando {wait_time_on_error_s}s...")
                        time.sleep(wait_time_on_error_s)
                    else:
                        break # Otros errores no reintentables

            # Calcular métricas para Gemma
            similitud_gemma = cosine_final(np.array(text_embedding_gemma(respuesta_modelo_gemma)), np.array(text_embedding_gemma(respuesta_humana))) if successful_gemma_response else 0.0
            coherencia_gemma = medir_coherencia(pregunta, respuesta_modelo_gemma) if successful_gemma_response else 0.0
            personalizacion_gemma = medir_personalizacion(perfil_usuario, respuesta_modelo_gemma) if successful_gemma_response else 0.0
            fluidez_gemma = medir_fluidez(respuesta_modelo_gemma) if successful_gemma_response else 0.0
            alucinacion_gemma = medir_alucinacion(respuesta_modelo_gemma, contexto_gemma_usado) if successful_gemma_response else 0.0

            df_respuestas_detalladas.append({
                "Modelo": modelo_banco2,
                "Prompt_Tipo": prompt_system, # Usamos el nombre del prompt
                "Pregunta": pregunta,
                "Respuesta_Humana": respuesta_humana,
                "Respuesta_Modelo": respuesta_modelo_gemma,
                "Similitud_Coseno": similitud_gemma,
                "Coherencia": coherencia_gemma,
                "Personalizacion": personalizacion_gemma,
                "Fluidez": fluidez_gemma,
                "Alucinacion": alucinacion_gemma,
                "Input_Tokens": input_tokens_gemma,
                "Output_Tokens": output_tokens_gemma,
                "API_Latency_s": api_latency_gemma,
                "Total_Exec_Time_s": total_exec_time_gemma,
                "Successful_Response": successful_gemma_response
            })
        else:
            print(f"Saltando evaluación de GEMINI para pregunta '{pregunta[:50]}...', cliente no configurado.")


df_respuestas_detalladas = pd.DataFrame(df_respuestas_detalladas)

# --- Cálculo de Métricas y Resumen Final ---
# df_res_successful = df_respuestas_detalladas[df_respuestas_detalladas['Successful_Response'] == True].copy()

df_respuestas_detalladas = df_respuestas_detalladas[
    (df_respuestas_detalladas['Successful_Response'] == True) &
    (df_respuestas_detalladas['Similitud_Coseno'] != 0) &
    (df_respuestas_detalladas['Coherencia'] != 0) &
    (df_respuestas_detalladas['Personalizacion'] != 0) &
    (df_respuestas_detalladas['Fluidez'] != 0) &
    (df_respuestas_detalladas['Alucinacion'] != 0)
].copy()
df_respuestas_detalladas.index = range(df_respuestas_detalladas.shape[0])

# Calcular las tasas
df_respuestas_detalladas["Tasa"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Similitud_Coseno"]>=umbrales[0] else 0, axis = 1)
df_respuestas_detalladas["Coh"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Coherencia"]>=umbrales[1] else 0, axis = 1)
df_respuestas_detalladas["Per"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Personalizacion"]>=umbrales[2] else 0, axis = 1)
df_respuestas_detalladas["Flu"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Fluidez"]>=umbrales[3] else 0, axis = 1)
df_respuestas_detalladas["Alu"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Alucinacion"]>=umbrales[4] else 0, axis = 1)
df_respuestas_detalladas["total"] = 1 # Para contar el total de filas

# Escalamiento de la fluidez
if np.mean(df_respuestas_detalladas["Fluidez"]>=umbrales[3])>=0.30:
    min_F = df_respuestas_detalladas["Fluidez"].min()
    max_F = df_respuestas_detalladas["Fluidez"].max()
    df_respuestas_detalladas["Fluidez"]  = (df_respuestas_detalladas["Fluidez"] - min_F) / (max_F - min_F)
    df_respuestas_detalladas["Flu"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Fluidez"]>=0.50 else 0, axis = 1)

else:
    min_F = 0
    max_F = 1

# Agrupar y calcular promedios
summary_ = df_respuestas_detalladas.groupby(["Modelo", "Prompt_Tipo"]).agg(
    Tasa=('Tasa', 'mean'),
    Coh=('Coh', 'mean'),
    Per=('Per', 'mean'),
    Flu=('Flu', 'mean'),
    Alu=('Alu', 'mean'),
    Total_Respuestas=('total', 'sum'),
    Respuestas_Exitosas=('Successful_Response', 'sum'), # Contar las exitosas
    avg_input_tokens=('Input_Tokens', 'mean'),
    avg_output_tokens=('Output_Tokens', 'mean'),
    avg_api_latency=('API_Latency_s', 'mean'),
    avg_total_execution_time=('Total_Exec_Time_s', 'mean')
).reset_index()

# Convertir tasas a porcentaje
for k in ["Tasa", "Coh", "Per", "Flu", "Alu"]:
    summary_[k] = 100 * summary_[k]

# --- Impresión de Resultados ---

print("\n" + "="*80)
print("--- Detalles de Respuestas Generadas (DataFrame Completo) ---".center(80))
print("="*80)

Configuración cargada exitosamente desde config.yaml
Cliente Azure OpenAI inicializado.
Cliente Gemini/Gemma configurado.
Cargando y procesando PDF...
Generando embeddings para GPT (Azure)...
Generando embeddings para Gemma...
Enviando solicitud al LLM para generar 10 pares QA...
Generación del LLM completada. Procesando pares QA...


The following layers were not sharded: encoder.layer.*.attention.self.query.bias, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query.weight, encoder.layer.*.attention.self.key.bias, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.value.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key.weight, encoder.layer.*.attention.output.LayerNorm.weight, embeddings.position_embeddings.weight, encoder.layer.*.attention.self.value.bias, pooler.dense.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, pooler.dense.weight, embeddings.token_type_embeddings.weight
The following layers were not sharded: encoder.layer.*.attentio


         --- Detalles de Respuestas Generadas (DataFrame Completo) ---          


In [2]:
df_respuestas_detalladas.head(3)

,Modelo,Prompt_Tipo,Pregunta,Respuesta_Humana,Respuesta_Modelo,Similitud_Coseno,Coherencia,Personalizacion,Fluidez,Alucinacion,...,Output_Tokens,API_Latency_s,Total_Exec_Time_s,Successful_Response,Tasa,Coh,Per,Flu,Alu,total
0,gpt-4o-mini_clasificacion-PQRS,Eres un Asistente Personal de Inteligencia Art...,¿Qué es LLMOps?,LLMOps (Large Language Model Operations) es un...,"LLMOps, que significa ""Large Language Model Op...",0.956428,0.748917,0.796899,0.500527,0.771195,...,207,6.591404,7.666058,True,1,0,1,1,1,1
1,models/gemini-2.5-flash,Eres un Asistente Personal de Inteligencia Art...,¿Qué es LLMOps?,LLMOps (Large Language Model Operations) es un...,"¡Hola! Con gusto te explico qué es LLMOps, que...",0.834858,0.768006,0.789986,0.746821,0.766108,...,425,12.073298,12.336439,True,1,0,1,1,1,1
2,gpt-4o-mini_clasificacion-PQRS,Eres un Asistente Personal de Inteligencia Art...,¿Cuáles son los objetivos principales de LLMOps?,LLMOps busca facilitar las capacidades de IA c...,"Los objetivos principales de LLMOps, que se re...",0.892234,0.795700,0.793326,0.351179,0.766315,...,391,3.521293,4.228223,True,1,0,1,0,1,1


In [3]:
cl = summary_.columns
summary_["Indice Monitoreo"] = 0.20 * (summary_["Tasa"] + summary_["Coh"] + summary_["Per"] + summary_["Flu"] + summary_["Alu"])
cl2 = [x for x in cl if x not in ["Modelo","Prompt_Tipo"]]
summary_ = summary_.get(["Modelo","Prompt_Tipo","Indice Monitoreo"] + cl2)
summary_

,Modelo,Prompt_Tipo,Indice Monitoreo,Tasa,Coh,Per,Flu,Alu,Total_Respuestas,Respuestas_Exitosas,avg_input_tokens,avg_output_tokens,avg_api_latency,avg_total_execution_time
0,gpt-4o-mini_clasificacion-PQRS,Eres un Asistente Personal de Inteligencia Art...,82.0,100.0,40.0,100.0,70.0,100.0,10,10,2904.6,342.1,4.634059,5.511642
1,gpt-4o-mini_clasificacion-PQRS,Eres una Inteligencia Artificial super avanzad...,72.0,90.0,60.0,100.0,10.0,100.0,10,10,2657.6,215.3,2.447868,3.198969
2,models/gemini-2.5-flash,Eres un Asistente Personal de Inteligencia Art...,72.0,40.0,30.0,100.0,90.0,100.0,10,10,2112.2,370.5,7.410099,7.754081
3,models/gemini-2.5-flash,Eres una Inteligencia Artificial super avanzad...,70.0,60.0,70.0,100.0,20.0,100.0,10,10,1951.2,127.4,3.273753,3.577786


In [4]:
summary_["GPT"] = summary_['Modelo'].str.contains(r'(?i)\bgpt\b', na=False).astype(int)

In [5]:
best_model = summary_.loc[(summary_["Indice Monitoreo"]==np.max(summary_["Indice Monitoreo"])),]
best_model.index = range(best_model.shape[0])

best_model = best_model.loc[best_model["avg_total_execution_time"]==np.min(best_model["avg_total_execution_time"]),]
best_model.index = range(best_model.shape[0])

model_ = best_model["Modelo"][0]
pmpt_ = best_model["Prompt_Tipo"][0]
ind_flow_gpt = summary_["GPT"][0]

## retornos

* summary_
* df_respuestas_detalladas
* min_F
* max_F
* model_
* pmpt_
* ind_flow_gpt
* df_vector_store_gpt
* df_vector_store_gem

## PR

* Creación de preguntas y respuestas

* Monitoring

## INPUTS

* contexto_para_generacion,
* num_pares_a_generar
* temp_
* model_
* pmpt_
* ind_flow_gpt
* df_vector_store_gpt
* df_vector_store_gem
* MD_GEN
* modelo_banco_generacion
* temperature_gen_assitant

In [6]:
modelo_banco_generacion

'models/gemini-1.5-pro-latest'

In [7]:
temperature_gen_assitant

0.7

In [8]:
num_pares_a_generar = 15

In [9]:
temp_ = 0.0

In [10]:
model_

'gpt-4o-mini_clasificacion-PQRS'

In [11]:
pmpt_

'Eres un Asistente Personal de Inteligencia Artificial experto en explicar información técnica de manera clara, concisa y fácil de entender para cualquier persona, incluso para un público no técnico. Tu objetivo es ser extremadamente útil y amigable.\n\nPara responder a las preguntas del usuario, **siempre debes basarte en los RESULTADOS DE BÚSQUEDA SEMÁNTICA proporcionados.**\n\nInstrucciones CLAVE para generar tu respuesta:\n1.  **Prioriza la Coherencia:** Asegúrate de que tu respuesta esté DIRECTAMENTE relacionada con la pregunta. No te desvíes del tema.\n2.  **Responde de manera fluida. Imagina que le estás explicando a alguien que no es experto en el tema. Puedes hacer una mezcla donde prime lo coloquial en el discurso sobre lo técnico.\n3.  **Mantén la Precisión y Veracidad:** Solo utiliza la información de la BÚSQUEDA SEMÁNTICA si es relevante y tiene relación directa con la pregunta.\n4.  **No Inventes (Evita Alucinaciones):** Si la respuesta no se encuentra explícitamente en e

In [12]:
ind_flow_gpt

np.int64(1)

In [12]:
# --- GENERACIÓN DE NUEVAS PREGUNTAS Y RESPUESTAS ---
nuevos_pares_qa = []

# Corregido: Usar 'top_join' para la generación de QA, que ahora proviene del YAML

if MD_GEN == "GPT":
    contexto_para_generacion = obtener_contexto_para_generacion(df_vector_store_gem, 
                                                            num_chunks=top_join)
else:
    contexto_para_generacion = obtener_contexto_para_generacion(df_vector_store_gpt, 
                                                            num_chunks=top_join)

# Formatear el prompt de generación con el número deseado (ahora `num_pares_a_generar` del YAML)
final_prompt_qa = prompt_generacion_qa.format(
    source_text=contexto_para_generacion,
    num_to_generate=num_pares_a_generar
)

print(f"Enviando solicitud al LLM para generar {num_pares_a_generar} pares QA...")
try:
    if client: # Solo intentar si el cliente Azure está inicializado
        if MD_GEN == "GPT":
            completion_qa = client.chat.completions.create(
            model=modelo_banco_generacion,
            temperature=temperature_gen_assitant, # Usa la temperatura de GPT del YAML
            messages=[
                {"role": "system", "content": final_prompt_qa}
            ]
            )
            generated_text = completion_qa.choices[0].message.content
        else:
            model_2 = genai.GenerativeModel(model_name=modelo_banco_generacion)
            full_query_for_gemma = final_prompt_qa
            completion2 = model_2.generate_content(
            contents=full_query_for_gemma,
            generation_config=GenerationConfig(temperature=temperature_gen_assitant)
            )
            generated_text = completion2.text


        print("Generación del LLM completada. Procesando pares QA...")

        # --- PARSEO DE LAS RESPUESTAS GENERADAS CON REGEX (PARA MÚLTIPLES PARES) ---
        pattern = re.compile(r"Pregunta:\s*(.*?)\s*Respuesta:\s*(.*?)(?=(?:Pregunta:|$))", re.DOTALL | re.IGNORECASE)
        matches = pattern.findall(generated_text)

        for match in matches:
            question = match[0].strip()
            answer = match[1].strip()

            if question and answer: # Asegurarse de que no estén vacíos
                # Validar si ya existe para evitar duplicados
                if not df_preguntas_respuestas_existentes['Pregunta'].astype(str).str.contains(question, case=False, regex=False).any():
                    nuevos_pares_qa.append({"Pregunta": question, "Respuesta": answer})
                else:
                    print(f"  [QA Gen] Par '{question[:30]}...' ya existe en 'preguntas_BOT.xlsx', omitiendo.")

                if len(nuevos_pares_qa) >= num_pares_a_generar:
                    break # Detener si ya generamos la cantidad deseada

    else:
        print("Error: El cliente Azure OpenAI no está inicializado. No se pudo generar QA.")

except Exception as e:
    print(f"Error durante la generación o procesamiento de QA: {e}")

# Asegurarse de tener a lo sumo num_pares_a_generar
if len(nuevos_pares_qa) > num_pares_a_generar:
    nuevos_pares_qa = nuevos_pares_qa[:num_pares_a_generar]
elif len(nuevos_pares_qa) < num_pares_a_generar:
    print(f"Advertencia: Solo se pudieron generar {len(nuevos_pares_qa)} pares QA válidos de {num_pares_a_generar} solicitados.")

df_nuevos_qa = pd.DataFrame(nuevos_pares_qa)

if df_nuevos_qa.empty:
    print("No se pudieron generar pares de preguntas y respuestas válidos. Finalizando.")
    exit()

Enviando solicitud al LLM para generar 15 pares QA...
Generación del LLM completada. Procesando pares QA...


In [16]:
df_nuevos_qa.head(5)

,Pregunta,Respuesta
0,¿Qué es LLMOps?,LLMOps (Large Language Model Operations) es un...
1,¿Cuál es el objetivo principal de LLMOps?,El objetivo de LLMOps es habilitar las capacid...
2,¿Por qué es importante LLMOps para los modelos...,LLMOps aborda los desafíos específicos de los ...
3,¿Cuáles son algunos de los desafíos que LLMOps...,Los LLMs requieren importantes recursos comput...
4,¿Por qué es preferible usar modelos de base pr...,Entrenar LLMs desde cero es costoso y requiere...


In [ ]:
df_respuestas_detalladas = []
for i, row in df_nuevos_qa.iterrows():
    if ind_flow_gpt==1:
        pregunta = str(row["Pregunta"])
        respuesta_humana = str(row["Respuesta"])

        respuesta_modelo_gpt = "No se pudo generar respuesta."
        input_tokens_gpt = 0
        output_tokens_gpt = 0
        api_latency_gpt = 0
        total_exec_time_gpt = 0
        successful_gpt_response = False
        contexto_gpt_usado = ""

        for attempt in range(max_attempts):
            try:
                start_time_total = time.time()
                context_chunks_gpt = get_context_from_query(pregunta, df_vector_store_gpt, top_join, text_embedding)
                contexto_gpt_usado = " ".join(context_chunks_gpt) # Para alucinación
                custom_prompt_gpt = pmpt_.format(source=str(context_chunks_gpt))

                llm_start_time = time.time()
                completion = client.chat.completions.create(
                        model=model_,
                        temperature=temp_,
                        messages=[
                            {"role": "system", "content": custom_prompt_gpt},
                            {"role": "user", "content": pregunta}
                        ]
                )
                respuesta_modelo_gpt = completion.choices[0].message.content
                llm_end_time = time.time()

                input_tokens_gpt = completion.usage.prompt_tokens
                output_tokens_gpt = completion.usage.completion_tokens
                api_latency_gpt = llm_end_time - llm_start_time
                total_exec_time_gpt = time.time() - start_time_total
                successful_gpt_response = True
                break # Salir del bucle de reintentos si fue exitoso
            except Exception as e:
                print(f"    [GPT] Error en intento {attempt + 1}/{max_attempts} para '{pregunta[:50]}...': {e}")
                if "quota" in str(e).lower() or "rate limit" in str(e).lower():
                    print(f"    [GPT] Error de cuota. Esperando {wait_time_on_error_s}s...")
                    time.sleep(wait_time_on_error_s)
                else:
                    break # Otros errores no reintentables

        # Calcular métricas para GPT
        similitud_gpt = cosine_final(np.array(text_embedding(respuesta_modelo_gpt)), np.array(text_embedding(respuesta_humana))) if successful_gpt_response else 0.0
        coherencia_gpt = medir_coherencia(pregunta, respuesta_modelo_gpt) if successful_gpt_response else 0.0
        personalizacion_gpt = medir_personalizacion(perfil_usuario, respuesta_modelo_gpt) if successful_gpt_response else 0.0
        fluidez_gpt = medir_fluidez(respuesta_modelo_gpt) if successful_gpt_response else 0.0
        alucinacion_gpt = medir_alucinacion(respuesta_modelo_gpt, contexto_gpt_usado) if successful_gpt_response else 0.0

        df_respuestas_detalladas.append({
                "Pregunta": pregunta,
                "Respuesta_Humana": respuesta_humana,
                "Respuesta_Modelo": respuesta_modelo_gpt,
                "Similitud_Coseno": similitud_gpt,
                "Coherencia": coherencia_gpt,
                "Personalizacion": personalizacion_gpt,
                "Fluidez": fluidez_gpt,
                "Alucinacion": alucinacion_gpt,
                "Input_Tokens": input_tokens_gpt,
                "Output_Tokens": output_tokens_gpt,
                "API_Latency_s": api_latency_gpt,
                "Total_Exec_Time_s": total_exec_time_gpt,
                "Successful_Response": successful_gpt_response
        })


        
    else:
        respuesta_modelo_gemma = "No se pudo generar respuesta."
        input_tokens_gemma = 0
        output_tokens_gemma = 0
        api_latency_gemma = 0
        total_exec_time_gemma = 0
        successful_gemma_response = False
        contexto_gemma_usado = ""

        for attempt in range(max_attempts):
            try:
                start_time_total = time.time()
                context_chunks_gemma = get_context_from_query(pregunta, df_vector_store_gem, top_join, text_embedding_gemma)
                contexto_gemma_usado = " ".join(context_chunks_gemma) # Para alucinación
                custom_prompt_gemma = pmpt_.format(source=str(context_chunks_gemma))
                model_2 = genai.GenerativeModel(model_name=model_)
                full_query_for_gemma = custom_prompt_gemma + "\n\nPregunta del usuario: " + pregunta

                llm_start_time = time.time()
                completion2 = model_2.generate_content(
                        contents=full_query_for_gemma,
                        generation_config=GenerationConfig(temperature=temp_)
                )
                respuesta_modelo_gemma = completion2.text
                llm_end_time = time.time()

                # Para Gemini/Gemma, los tokens son una estimación simple si no se exponen directamente en la respuesta
                input_tokens_gemma = len(full_query_for_gemma.split())
                output_tokens_gemma = len(respuesta_modelo_gemma.split())
                api_latency_gemma = llm_end_time - llm_start_time
                total_exec_time_gemma = time.time() - start_time_total
                successful_gemma_response = True
                break # Salir del bucle de reintentos si fue exitoso
            except Exception as e:
                print(f"    [GEMINI] Error en intento {attempt + 1}/{max_attempts} para '{pregunta[:50]}...': {e}")
                if "ResourceExhausted" in str(e) or "quota" in str(e).lower() or "rate limit" in str(e).lower():
                    print(f"    [GEMINI] Error de cuota. Esperando {wait_time_on_error_s}s...")
                    time.sleep(wait_time_on_error_s)
                else:
                    break # Otros errores no reintentables

        # Calcular métricas para Gemma
        similitud_gemma = cosine_final(np.array(text_embedding_gemma(respuesta_modelo_gemma)), np.array(text_embedding_gemma(respuesta_humana))) if successful_gemma_response else 0.0
        coherencia_gemma = medir_coherencia(pregunta, respuesta_modelo_gemma) if successful_gemma_response else 0.0
        personalizacion_gemma = medir_personalizacion(perfil_usuario, respuesta_modelo_gemma) if successful_gemma_response else 0.0
        fluidez_gemma = medir_fluidez(respuesta_modelo_gemma) if successful_gemma_response else 0.0
        alucinacion_gemma = medir_alucinacion(respuesta_modelo_gemma, contexto_gemma_usado) if successful_gemma_response else 0.0

        df_respuestas_detalladas.append({
                "Pregunta": pregunta,
                "Respuesta_Humana": respuesta_humana,
                "Respuesta_Modelo": respuesta_modelo_gemma,
                "Similitud_Coseno": similitud_gemma,
                "Coherencia": coherencia_gemma,
                "Personalizacion": personalizacion_gemma,
                "Fluidez": fluidez_gemma,
                "Alucinacion": alucinacion_gemma,
                "Input_Tokens": input_tokens_gemma,
                "Output_Tokens": output_tokens_gemma,
                "API_Latency_s": api_latency_gemma,
                "Total_Exec_Time_s": total_exec_time_gemma,
                "Successful_Response": successful_gemma_response
        })


df_respuestas_detalladas = pd.DataFrame(df_respuestas_detalladas)

# --- Cálculo de Métricas y Resumen Final ---
# df_res_successful = df_respuestas_detalladas[df_respuestas_detalladas['Successful_Response'] == True].copy()

df_respuestas_detalladas = df_respuestas_detalladas[
    (df_respuestas_detalladas['Successful_Response'] == True) &
    (df_respuestas_detalladas['Similitud_Coseno'] != 0) &
    (df_respuestas_detalladas['Coherencia'] != 0) &
    (df_respuestas_detalladas['Personalizacion'] != 0) &
    (df_respuestas_detalladas['Fluidez'] != 0) &
    (df_respuestas_detalladas['Alucinacion'] != 0)
].copy()
df_respuestas_detalladas.index = range(df_respuestas_detalladas.shape[0])

# Calcular las tasas
df_respuestas_detalladas["Tasa"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Similitud_Coseno"]>=umbrales[0] else 0, axis = 1)
df_respuestas_detalladas["Coh"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Coherencia"]>=umbrales[1] else 0, axis = 1)
df_respuestas_detalladas["Per"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Personalizacion"]>=umbrales[2] else 0, axis = 1)
df_respuestas_detalladas["Flu"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Fluidez"]>=umbrales[3] else 0, axis = 1)
df_respuestas_detalladas["Alu"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Alucinacion"]>=umbrales[4] else 0, axis = 1)
df_respuestas_detalladas["total"] = 1 # Para contar el total de filas
df_respuestas_detalladas["Modelo"] = model_

The following layers were not sharded: encoder.layer.*.attention.self.query.bias, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query.weight, encoder.layer.*.attention.self.key.bias, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.value.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key.weight, encoder.layer.*.attention.output.LayerNorm.weight, embeddings.position_embeddings.weight, encoder.layer.*.attention.self.value.bias, pooler.dense.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, pooler.dense.weight, embeddings.token_type_embeddings.weight
The following layers were not sharded: encoder.layer.*.attentio

In [20]:
if (min_F!=0) & (max_F!=1):
    df_respuestas_detalladas["Fluidez2"]  = (df_respuestas_detalladas["Fluidez"] - min_F) / (max_F - min_F)
    df_respuestas_detalladas["Flu2"] = df_respuestas_detalladas.apply(lambda row: 1 if row["Fluidez2"]>=0.50 else 0, axis = 1)

In [21]:
# Agrupar y calcular promedios
summary_ = df_respuestas_detalladas.groupby(["Modelo"]).agg(
    Tasa=('Tasa', 'mean'),
    Coh=('Coh', 'mean'),
    Per=('Per', 'mean'),
    Flu=('Flu', 'mean'),
    Flu2=('Flu2', 'mean'),
    Alu=('Alu', 'mean'),
    Total_Respuestas=('total', 'sum'),
    Respuestas_Exitosas=('Successful_Response', 'sum'), # Contar las exitosas
    avg_input_tokens=('Input_Tokens', 'mean'),
    avg_output_tokens=('Output_Tokens', 'mean'),
    avg_api_latency=('API_Latency_s', 'mean'),
    avg_total_execution_time=('Total_Exec_Time_s', 'mean')
).reset_index()

# Convertir tasas a porcentaje
for k in ["Tasa", "Coh", "Per", "Flu", "Flu2" ,"Alu"]:
    summary_[k] = 100 * summary_[k]

summary_

,Modelo,Tasa,Coh,Per,Flu,Flu2,Alu,Total_Respuestas,Respuestas_Exitosas,avg_input_tokens,avg_output_tokens,avg_api_latency,avg_total_execution_time
0,gpt-4o-mini_clasificacion-PQRS,100.0,86.666667,100.0,13.333333,20.0,100.0,15,15,2736.733333,226.333333,2.778824,3.261617


In [22]:
## Métricas FInales Monitoring:
fl = np.max([summary_["Flu"][0].round(3),summary_["Flu2"][0].round(3)])

print("Tasa de respuestas correctas: ",summary_["Tasa"][0].round(3),"%")
print("Tasa de coherencia: ",summary_["Coh"][0].round(3),"%")
print("Tasa de personalización: ",summary_["Per"][0].round(3),"%")
print("Tasa de fluidez: ",  fl  ,"%")
print("Tasa de no alucinación: ",summary_["Alu"][0].round(3),"%")

print("Tasa de precisión:" , 1/3 * (summary_["Tasa"][0].round(3) + summary_["Per"][0].round(3) + summary_["Alu"][0].round(3) ) ,"%" )
print("Tasa de calidad linguística:", 1/2 * (fl +  summary_["Coh"][0].round(3) ),"%" )

print("Promedio de tokens en input: ",summary_["avg_input_tokens"][0].round(3))
print("Promedio de tokens en output: ",summary_["avg_output_tokens"][0].round(3))
print("Promedio de latencia API (seg): ",summary_["avg_api_latency"][0].round(3))
print("Promedio de respuesta (seg): ",summary_["avg_total_execution_time"][0].round(3))

Tasa de respuestas correctas:  100.0 %
Tasa de coherencia:  86.667 %
Tasa de personalización:  100.0 %
Tasa de fluidez:  20.0 %
Tasa de no alucinación:  100.0 %
Tasa de precisión: 100.0 %
Tasa de calidad linguística: 53.3335 %
Promedio de tokens en input:  2736.733
Promedio de tokens en output:  226.333
Promedio de latencia API (seg):  2.779
Promedio de respuesta (seg):  3.262


## BATCH CUSTOM PREDICT

In [23]:
batch = pd.read_excel("CRONBACH.xlsx")
batch

,Pregunta
0,¿Qué son los modelos de lenguaje de dominio es...
1,¿Cuál es la importancia de los términos de lic...
2,¿Qué es un entorno de sandbox en el contexto d...
3,¿Cuál es el propósito de la evaluación de recu...
4,¿Qué es la evaluación de benchmarks en LLMs?
...,...
63,¿Qué es el modelo Koala y cuáles son sus carac...
64,¿Cómo se evalúan los modelos LLM para medir su...
65,¿Qué papel juegan las evaluaciones de sesgo en...
66,¿Cuál es el objetivo del uso de modelos de IA ...


In [24]:
if ind_flow_gpt == 1:
    df_vector_ = df_vector_store_gpt.copy()
else:
    df_vector_ = df_vector_store_gem.copy()

In [28]:
def model_response(pregunta,ind_flow_gpt,top_join,df_vector_,pmpt_,model_,temp_):
    if ind_flow_gpt==1:
        context_chunks_gpt = get_context_from_query(pregunta, df_vector_, top_join, text_embedding)
        custom_prompt_gpt = pmpt_.format(source=str(context_chunks_gpt))

        completion = client.chat.completions.create(
                        model=model_,
                        temperature=temp_,
                        messages=[
                            {"role": "system", "content": custom_prompt_gpt},
                            {"role": "user", "content": pregunta}
                        ]
        )
        respuesta_modelo = completion.choices[0].message.content
    else:
        context_chunks_gemma = get_context_from_query(pregunta, df_vector_, top_join, text_embedding_gemma)
        custom_prompt_gemma = pmpt_.format(source=str(context_chunks_gemma))
        model_2 = genai.GenerativeModel(model_name=model_)
        full_query_for_gemma = custom_prompt_gemma + "\n\nPregunta del usuario: " + pregunta

        completion2 = model_2.generate_content(
                        contents=full_query_for_gemma,
                        generation_config=GenerationConfig(temperature=temp_)
        )
        respuesta_modelo = completion2.text
    return respuesta_modelo

In [36]:
respuestas_ = []

for i, row in batch.iterrows(): 
    pregunta = str(row["Pregunta"])
    respuesta = model_response(pregunta = pregunta,ind_flow_gpt = ind_flow_gpt,top_join = top_join,df_vector_ = df_vector_,pmpt_ = pmpt_,model_ = model_,temp_ = temp_)
    respuestas_.append(respuesta)

In [38]:
batch["Respuesta"] = respuestas_

In [42]:
batch.head(10)

,Pregunta,Respuesta
0,¿Qué son los modelos de lenguaje de dominio es...,Los modelos de lenguaje de dominio específico ...
1,¿Cuál es la importancia de los términos de lic...,La importancia de los términos de licencia en ...
2,¿Qué es un entorno de sandbox en el contexto d...,Un entorno de sandbox en el contexto de LLMOps...
3,¿Cuál es el propósito de la evaluación de recu...,La evaluación de recuperación en el uso de Mod...
4,¿Qué es la evaluación de benchmarks en LLMs?,La evaluación de benchmarks en modelos de leng...
5,¿Qué es el modelo Vicuna y cuál es su relación...,Vicuna es un modelo de lenguaje que se inspira...
6,¿Cómo se define un entorno de producción en LL...,Un entorno de producción en LLMOps es el espac...
7,¿Qué pasos son esenciales antes de iniciar un ...,Antes de iniciar un proyecto de LLM (Modelos d...
8,¿Qué es CHAIR en la evaluación de modelos de l...,"CHAIR, que significa ""Caption Hallucination As..."
9,¿Por qué es importante la democratización de l...,La democratización de la tecnología de intelig...


In [43]:
batch.to_excel("Mail_Notification.xlsx",index=False)